Download Model from HuggingFace

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
from huggingface_hub import snapshot_download

model_name = "coderop12/Legal_Summarization_Model"
base_model = "./original_model/"
snapshot_download(repo_id=model_name, local_dir=base_model, ignore_patterns=["*.pth"])

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

'/content/original_model'

Clone llama.cpp Repository

In [3]:
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 41152, done.
remote: Counting objects: 100% (7013/7013), done.
remote: Compressing objects: 100% (392/392), done.
remote: Total 41152 (delta 6793), reused 6624 (delta 6621), pack-reused 34139 (from 2)
Receiving objects: 100% (41152/41152), 71.54 MiB | 25.55 MiB/s, done.
Resolving deltas: 100% (30010/30010), done.


In [4]:
!mkdir models

Convert model to GGUF format

In [5]:
!python llama.cpp/convert_hf_to_gguf.py ./original_model/ --outfile models/llama_3.1_FP16.gguf

INFO:hf-to-gguf:Loading model: original_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:output.weight,               torch.bfloat16 --> F16, shape = {2048, 32000}
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> F16, shape = {2048, 32000}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.bfloat16 --> F16, shape = {5632, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.bfloat16 --> F16, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.bfloat16 --> F16, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.bfloat16 --> F16, shape = {2048, 256}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.bfloat16 --> F16, shape = {20

Build llama.cpp and quantize the Model

In [6]:
!mkdir llama.cpp/build && cd llama.cpp/build && cmake .. && cmake --build . --config Release

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- Including CPU backend
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found OpenMP_CXX: -fopenmp (found version "4.5")
-- Found OpenMP: TRUE (found version "4.5

In [7]:
!cd llama.cpp/build/bin && ./llama-quantize /content/models/llama_3.1_FP16.gguf /content/models/llama_3.1-Q4_K_M.gguf q4_K_M

main: build = 4462 (c05e8c99)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '/content/models/llama_3.1_FP16.gguf' to '/content/models/llama_3.1-Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 35 key-value pairs and 201 tensors from /content/models/llama_3.1_FP16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = TinyLlama 1.1B Chat v1.0
llama_model_loader: - kv   3:                            general.version str              = v1.0
llama_model_loader: - kv   4:                       general.organization str              = TinyLlama
llama_model_loader: - kv   5

Now Inference using Quantized Model

In [ ]:
!pip install llama-cpp-python==0.2.85

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 MB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00


In [ ]:
from llama_cpp import Llama

In [ ]:
model_path = "/content/models/llama_3.1-Q4_K_M.gguf"

In [ ]:
llm = Llama(model_path=model_path)

llama_model_loader: loaded meta data with 26 key-value pairs and 292 tensors from /content/models/llama_3.1-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Original_Model
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3.1
llama_model_loader: - kv   5:                               general.tags arr[str,6]       = ["facebook", "meta", "pytorch", "llam...
llama_model_loader: - kv   6:                          general.languages arr[str,8]       = ["en", "de", "fr", "it", "pt"

In [ ]:
generation_kwargs = {
    "max_tokens":200,
    "echo":False,
    "top_k":1
}

prompt = "Which country hosted 2018 fifa world cup?"
res = llm(prompt, **generation_kwargs)
res


llama_print_timings:        load time =    5582.23 ms
llama_print_timings:      sample time =      22.08 ms /   200 runs   (    0.11 ms per token,  9056.33 tokens per second)
llama_print_timings: prompt eval time =    5582.13 ms /    12 tokens (  465.18 ms per token,     2.15 tokens per second)
llama_print_timings:        eval time =  152621.22 ms /   199 runs   (  766.94 ms per token,     1.30 tokens per second)
llama_print_timings:       total time =  158636.78 ms /   211 tokens


{'id': 'cmpl-edb6eac3-82b3-4880-91d8-30c17069b277',
 'object': 'text_completion',
 'created': 1723112699,
 'model': '/content/models/llama_3.1-Q4_K_M.gguf',
 'choices': [{'text': " The 2018 fifa world cup was the 21st fifa world cup, an international football tournament contested by the men's national teams of the member associations of fifa. It took place in russia from 14 june to 15 july 2018. It was the first world cup to be held in eastern europe, and the 11th time that it had been held in europe. At an estimated cost of over $14.2 billion, it. The 2018 fifa world cup was the 21st fifa world cup, an international football tournament contested by the men's national teams of the member associations of fifa. It took place in russia from 14 june to 15 july 2018. It was the first world cup to be held in eastern europe, and the 11th time that it had been held in europe. At an estimated cost of over $14.2 billion, it. The 2018",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'len

Save model to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir "/content/drive/My Drive/llama_models"

In [ ]:
import shutil

source_file_path = '/content/models/llama_3.1-Q4_K_M.gguf'
destination_file_path = '/content/drive/My Drive/llama_models/llama_3.1-Q4_K_M.gguf'

shutil.copy(source_file_path, destination_file_path)

Upload model to Huggingface Hub

In [ ]:
from huggingface_hub import login
login('your_token')

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

model_id = "atulkrishna/llama3.1-Q4_K_M-gguf"
api.create_repo(model_id, exist_ok=True, repo_type="model")
api.upload_file(
    path_or_fileobj='/content/models/llama_3.1-Q4_K_M.gguf',
    path_in_repo="llama3.1-Q4_K_M.gguf",
    repo_id=model_id,
)

Inference by utilizing GPU

In [ ]:
from huggingface_hub import snapshot_download

model_name = "atulkrishna/llama3.1-Q4_K_M-gguf"
base_model = "./quantized_models/"
snapshot_download(repo_id=model_name, local_dir=base_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

llama3.1-Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

'/content/quantized_models'

In [ ]:
!nvidia-smi

Thu Aug  8 11:23:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00


In [ ]:
from llama_cpp import Llama
model_path = "./quantized_models/llama3.1-Q4_K_M.gguf"

model = Llama(model_path=model_path, n_gpu_layers=-1)

llama_model_loader: loaded meta data with 26 key-value pairs and 292 tensors from ./quantized_models/llama3.1-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Original_Model
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3.1
llama_model_loader: - kv   5:                               general.tags arr[str,6]       = ["facebook", "meta", "pytorch", "llam...
llama_model_loader: - kv   6:                          general.languages arr[str,8]       = ["en", "de", "fr", "it", "p

In [ ]:
generation_kwargs = {
    "max_tokens":200,
    "echo":False,
    "top_k":1
}

prompt = "Which country hosted 2018 fifa world cup?"
res = model(prompt, **generation_kwargs)
res

Llama.generate: prefix-match hit

llama_print_timings:        load time =     545.44 ms
llama_print_timings:      sample time =      16.89 ms /   200 runs   (    0.08 ms per token, 11842.03 tokens per second)
llama_print_timings: prompt eval time =      67.41 ms /     9 tokens (    7.49 ms per token,   133.52 tokens per second)
llama_print_timings:        eval time =    5408.86 ms /   199 runs   (   27.18 ms per token,    36.79 tokens per second)
llama_print_timings:       total time =    5691.39 ms /   208 tokens


{'id': 'cmpl-94cf14c4-8b6e-4861-a2c8-0eec891a92c2',
 'object': 'text_completion',
 'created': 1723116609,
 'model': './quantized_models/llama3.1-Q4_K_M.gguf',
 'choices': [{'text': " The 2018 fifa world cup was the 21st fifa world cup, an international football tournament contested by the men's national teams of the member associations of fifa. It took place in russia from 14 june to 15 july 2018. It was the first world cup to be held in eastern europe, and the 11th time that it had been held in europe. At an estimated cost of over $14.2 billion, it. The 2018 fifa world cup was the 21st fifa world cup, an international football tournament contested by the men's national teams of the member associations of fifa. It took place in russia from 14 june to 15 july 2018. It was the first world cup to be held in eastern europe, and the 11th time that it had been held in europe. At an estimated cost of over $14.2 billion, it. The 2018",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'l

In [ ]:
output = model("Provide Information about world war 2 in 1000 words.", max_tokens=2048, stop=["\n"], echo=False)

In [ ]:
print(output['choices'][0]['text'])